TRATAMENTO DE DADOS BRUTOS VIA PYTHON PARA GERAÇÃO DE ARQUIVO CSV E POSTERIOR INGESTÃO DE DADOS NO POSTGRESQL.
O CÓDIGO DAS CÉLULAS FORAM CRIADOS DE FORMA TOTALMENTE INDEPENDENTE UNS DOS OUTROS.

In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


PARA REALIZAR O ETL DA TABELA BANCOS FOI UTILIZADA A TABELA DE DADOS BRUTOS BANCOS.CSV

In [2]:
#CÓDIGO USADO PARA ETL TABELA BANCOS

import pandas as pd

# Leitura da tabela
df = pd.read_csv(r'C:\\poli\\Ingestão\\trabalho 2\\Dados Brutos\\Python\\Bancos.csv')

# 1. Mover as linhas que contêm 'Prudencial' na coluna Nome para o topo
df_prudencial = df[df['Nome'].str.contains('Prudencial', case=False, na=False)]
df_outros = df[~df['Nome'].str.contains('Prudencial', case=False, na=False)]
df_ordenado = pd.concat([df_prudencial, df_outros], ignore_index=True)

# 2. Remover as primeiras ocorrências de CNPJ duplicados (mantendo a última)
df_final = df_ordenado.drop_duplicates(subset='CNPJ', keep='last')

#remove PRUDENCIAL dos demais como BTG Prudencial
df_final['NOME_SEM_PRUDENCIAL'] = df_final['Nome'].str.replace(' - PRUDENCIAL', '')

# Salva o DataFrame final em um novo arquivo CSV (ou sobrescreve o existente)
df_final.to_csv(r'C:\\poli\\Ingestão\\trabalho 2\\Dados Brutos\\Python\\ETL_tabela_bancos.csv', index=False)

print("Linhas movidas, duplicatas removidas e texto 'PRUDENCIAL' excluído com sucesso!")


Linhas movidas, duplicatas removidas e texto 'PRUDENCIAL' excluído com sucesso!


C:\Users\leander.silveira\AppData\Local\Temp\ipykernel_20792\1205791932.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['NOME_SEM_PRUDENCIAL'] = df_final['Nome'].str.replace(' - PRUDENCIAL', '')


PARA REALIZAR O ETL DA TABELA EMPREGADOS FORAM UTILIZADAS AS TABELAS DE DADOS glassdoor_consolidado_join_match_less_v2_renomeado.csv E glassdoor_consolidado_join_match_v2_renomeado.csv. OS CSVs ESTÃO COM O FINAL _RENOMEADOS, POIS TEVE A CADIFICAÇÃO DE CARACTER CORRIGIDA PARA UTH-8 NO NOTPAD++.

In [1]:
#CÓDIGO USADO PARA ETL TABELA EMPREGADOS

import pandas as pd

# Definir os caminhos dos arquivos CSV
caminho_arquivo_csv1 = 'C:/poli/Ingestão/trabalho 2/Dados Brutos/Python/glassdoor_consolidado_join_match_less_v2_renomeado.csv'
caminho_arquivo_csv2 = 'C:/poli/Ingestão/trabalho 2/Dados Brutos/Python/glassdoor_consolidado_join_match_v2_renomeado.csv'
caminho_arquivo_csv_resultado = 'C:/poli/Ingestão/trabalho 2/Dados Brutos/Python/ETL_tabela_empregados.csv'

# Ler os arquivos CSV
try:
    df1 = pd.read_csv(caminho_arquivo_csv1)
    df2 = pd.read_csv(caminho_arquivo_csv2)
    print("Arquivos lidos com sucesso.")
except Exception as e:
    print(f"Erro ao ler os arquivos CSV: {e}")
    exit()

# Verificar os nomes das colunas para garantir que correspondem
colunas_esperadas = [
    'employer_name', 'reviews_count', 'culture_count', 'salaries_count', 
    'benefits_count', 'employer-website', 'employer-headquarters', 
    'employer-founded', 'employer-industry', 'employer-revenue', 'url', 
    'Geral', 'Cultura e valores', 'Diversidade e inclusão', 'Qualidade de vida', 
    'Alta liderança', 'Remuneração e benefícios', 'Oportunidades de carreira', 
    'Recomendam para outras pessoas(%)', 'Perspectiva positiva da empresa(%)', 
    'CNPJ_Segmento', 'Nome', 'match_percent'
]

# Verificar se ambas as DataFrames têm as colunas esperadas
if set(colunas_esperadas).issubset(df1.columns) and set(colunas_esperadas).issubset(df2.columns):
    print("Ambos os arquivos têm as colunas esperadas.")
else:
    print("Os arquivos CSV não têm as colunas esperadas.")
    print(f"Colunas no primeiro arquivo: {df1.columns}")
    print(f"Colunas no segundo arquivo: {df2.columns}")
    exit()

# Concatenar os DataFrames
df_concatenado = pd.concat([df1, df2], ignore_index=True)

# Renomear as colunas conforme solicitado
novos_nomes_colunas = {
    'employer-website': 'employer_website',
    'employer-headquarters': 'employer_headquarters',
    'employer-founded': 'employer_founded',
    'employer-industry': 'employer_industry',
    'employer-revenue': 'employer_revenue',
    'Cultura e valores': 'cultura_e_valores',
    'Diversidade e inclusão': 'diversidade_e_inclusao',
    'Qualidade de vida': 'qualidade_de_vida',
    'Alta liderança': 'alta_lideranca',
    'Remuneração e benefícios': 'remuneracao_e_beneficios',
    'Oportunidades de carreira': 'oportunidades_de_carreira',
    'Recomendam para outras pessoas(%)': 'recomendam_para_outras_pessoas_PCT',
    'Perspectiva positiva da empresa(%)': 'perspectiva_positiva_da_empresa_PCT'
}

df_concatenado.rename(columns=novos_nomes_colunas, inplace=True)

# Salvar o DataFrame concatenado em um novo arquivo CSV
try:
    df_concatenado.to_csv(caminho_arquivo_csv_resultado, index=False)
    print(f"Arquivo CSV concatenado salvo como {caminho_arquivo_csv_resultado}")
except Exception as e:
    print(f"Erro ao salvar o arquivo CSV concatenado: {e}")


Arquivos lidos com sucesso.
Ambos os arquivos têm as colunas esperadas.
Arquivo CSV concatenado salvo como C:/poli/Ingestão/trabalho 2/Dados Brutos/Python/ETL_tabela_empregados.csv


PARA REALIZAR O ETL DA TABELA RECLAMAÇÕES FORAM UTILIZADAS AS TABELA DE DADOS BRUTOS PRESENTES EM PASTA ESPECÍFICA. 

In [3]:
#CÓDIGO USADO PARA ETL TABELA BANCOS

import pandas as pd
import os

# Defina o caminho da pasta onde os arquivos .csv estão localizados
pasta = "C:/poli/Ingestão/trabalho 2/Dados Brutos/Python/Reclamacoes"

# Lista para armazenar os DataFrames
dataframes = []

# Contador para garantir que apenas 8 arquivos sejam processados
contador = 0
max_arquivos = 8

# Itera sobre todos os arquivos na pasta
for file in os.listdir(pasta):
    if file.endswith('.csv') and contador < max_arquivos:
        caminho_arquivo = os.path.join(pasta, file)
        try:
            # Leia o arquivo CSV usando ponto e vírgula como delimitador
            df_loop = pd.read_csv(caminho_arquivo, sep=";", encoding="latin-1")
            
            # Adiciona o DataFrame à lista
            dataframes.append(df_loop)
            contador += 1
            print(f"Arquivo lido com sucesso: {file}")
        except Exception as e:
            print(f'Ocorreu um erro na leitura do arquivo {file}: {e}')

# Verifica se há DataFrames para concatenar
if dataframes:
    try:
        # Concatene todos os DataFrames em um único DataFrame
        df_concatenado = pd.concat(dataframes, ignore_index=True)
        
        # Exibir as colunas para verificar possíveis discrepâncias
        print("Colunas antes da renomeação:")
        print(df_concatenado.columns.tolist())
        
        # Definir o mapeamento de renomeação das colunas
        colunas_novas = [
            "ano"
            ,"trimestre"
            ,"categoria"
            ,"tipo"
            ,"CNPJ_IF"
            ,"instituicao_financeira"
            ,"indice"
            ,"quantidade_de_reclamacoes_reguladas_procedentes"
            ,"quantidade_de_reclamacoes_reguladas_outras"
            ,"quantidade_de_reclamacoes_nao_reguladas"
            ,"quantidade_total_de_reclamacoes"
            ,"quantidade_total_de_clientes_CCS_e_SCR"
            ,"quantidade_de_clientes_CCS"
            ,"quantidade_de_clientes_SCR"
            ,"Unnamed: 14"
        ]

        df_concatenado.columns = colunas_novas

        # Renomear as colunas, tratando possíveis erros de renomeação
    #    df_concatenado.rename(columns=novos_nomes_colunas, inplace=True)

        df_concatenado.drop('Unnamed: 14',axis=1,inplace=True)

        # Exibir as colunas após renomeação para verificação
        print("Colunas após renomeação:")
        print(df_concatenado.columns.tolist())
        


 #       df_concatenado['quantidade_total_de_clientes_CCS_e_SCR'] = df_concatenado['quantidade_total_de_clientes_CCS_e_SCR'].astype(int)
 #       df_concatenado['quantidade_total_de_clientes_CCS'] = df_concatenado['quantidade_total_de_clientes_CCS_e_SCR'].astype(int)
 #       df_concatenado['quantidade_total_de_clientes_SCR'] = df_concatenado['quantidade_total_de_clientes_CCS_e_SCR'].astype(int)

        df_concatenado.info()

        # Caminho para salvar o novo arquivo CSV concatenado
        caminho_saida = os.path.join(pasta, "ETL_tabela_reclamacoes.csv")
        
        df_concatenado.info()

        # Exporte o DataFrame concatenado para um novo arquivo CSV com cabeçalho
        df_concatenado.to_csv(caminho_saida, index=False, sep=";", encoding="latin-1")
        print(f"Arquivo CSV concatenado e renomeado salvo como {caminho_saida}")
    except Exception as e:
        print(f'Ocorreu um erro na concatenação ou salvamento do arquivo: {e}')
else:
    print("Nenhum arquivo CSV foi lido.")


Arquivo lido com sucesso: 2021_tri_01.csv
Arquivo lido com sucesso: 2021_tri_02.csv
Arquivo lido com sucesso: 2021_tri_03.csv
Arquivo lido com sucesso: 2021_tri_04.csv
Arquivo lido com sucesso: 2022_tri_01.csv
Ocorreu um erro na leitura do arquivo 2022_tri_02_nao_ha_dados.csv: No columns to parse from file
Arquivo lido com sucesso: 2022_tri_03.csv
Arquivo lido com sucesso: 2022_tri_04.csv
Colunas antes da renomeação:
['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF', 'Instituição financeira', 'Índice', 'Quantidade de reclamações reguladas procedentes', 'Quantidade de reclamações reguladas - outras', 'Quantidade de reclamações não reguladas', 'Quantidade total de reclamações', 'Quantidade total de clientes \x96 CCS e SCR', 'Quantidade de clientes \x96 CCS', 'Quantidade de clientes \x96 SCR', 'Unnamed: 14']
Colunas após renomeação:
['ano', 'trimestre', 'categoria', 'tipo', 'CNPJ_IF', 'instituicao_financeira', 'indice', 'quantidade_de_reclamacoes_reguladas_procedentes', 'quantidade_de_r